<a href="https://colab.research.google.com/github/MarinaSupiot/projet_7/blob/main/MLFLOW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import gc
from hyperopt import hp
from hyperopt import tpe
from hyperopt import fmin
from hyperopt import space_eval, STATUS_OK
import plotly.express as px
from plotly.offline import iplot

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

import lightgbm as lgb
#!pip install lime
#from lime import lime_tabular

import matplotlib as mpl
%matplotlib inline
import matplotlib.pyplot as plt
import missingno as msno
import numpy as np
import pandas as pd
import pickle
import random
import re
import seaborn as sns
!pip install shap
import shap


import sklearn
from sklearn.dummy import DummyClassifier
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler


from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, fbeta_score

from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

!pip install scikit-plot
!pip install kds

import scikitplot as skplt
import kds

import pickle
import os



import time

# Versions
print('Version des librairies utilisées :')


print('Matplotlib            : ' + mpl.__version__)
print('Missingno             : ' + msno.__version__)
print('NumPy                 : ' + np.version.full_version)
print('Pandas                : ' + pd.__version__)
print('Seaborn               : ' + sns.__version__)
print('Shap                  : ' + shap.__version__)
print('Sklearn               : ' + sklearn.__version__ )

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.9/532.9 kB 6.9 MB/s eta 0:00:00
Version des librairies utilisées :
Matplotlib            : 3.7.1
Missingno             : 0.5.2
NumPy                 : 1.23.5
Pandas                : 1.5.3
Seaborn               : 0.12.2
Shap                  : 0.43.0
Sklearn               : 1.2.2


In [37]:
def cout_metier(y_true, y_pred, fn_value=-10, fp_value=-1, vp_value=0, vn_value=1):
    mat_conf = confusion_matrix(y_true, y_pred)
    vn = mat_conf[0, 0]
    fn = mat_conf[1, 0]
    fp = mat_conf[0, 1]
    vp = mat_conf[1, 1]
    J = vp * vp_value + vn * vn_value + fp * fp_value + fn * fn_value
    # Coût maximum
    max_J = (fp + vn)*vn_value + (fn + vp)*vp_value

    # Coût minimum
    min_J = (fp + vn)*fp_value + (fn + vp)*fn_value

    # Coût normalisé entre 0 et 1
    J_normalized = (J - min_J)/(max_J - min_J)

    return J_normalized

Connexion MLFLOW

In [3]:
!pip install mlflow
import mlflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.0/19.0 MB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.6/230.6 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.1/148.1 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.8 MB/s eta 0:00:00


In [5]:
from sklearn.ensemble import AdaBoostClassifier

In [6]:
seed = 42

In [7]:
#chargement des train set et val set
train_set = pickle.load( open( "/content/drive/MyDrive/Projet 7/train_set.p", "rb" ) )
val_set = pickle.load( open( "/content/drive/MyDrive/Projet 7/val_set.p", "rb" ) )
train_labels = pickle.load( open( "/content/drive/MyDrive/Projet 7/train_labels.p", "rb" ) )
val_labels = pickle.load( open( "/content/drive/MyDrive/Projet 7/val_labels.p", "rb" ) )


In [8]:
train_sample_set = pickle.load( open( "/content/drive/MyDrive/Projet 7/train_sample_set.p", "rb" ) )
val_sample_set = pickle.load( open( "/content/drive/MyDrive/Projet 7/val_sample_set.p", "rb" ) )
train_sample_labels = pickle.load( open( "/content/drive/MyDrive/Projet 7/train_sample_labels.p", "rb" ) )
val_sample_labels = pickle.load( open( "/content/drive/MyDrive/Projet 7/val_sample_labels.p", "rb" ) )

In [41]:
# Définition d'une expérience (Une expérience peut contenir plusieurs *runs*)
mlflow.set_experiment("AdaBoost")
mlflow.sklearn.autolog()  # Ici on fait appel à l'API MLflow qui intègre les algos de Scikit-Learn.
                          # C'est grâce à ce code qu'on enregistre toutes les métriques et métadonnées de cette expérience (Module MLflow Tracking)

with open('/content/drive/MyDrive/Projet 7/Adaboost/model_smote.pkl', 'rb') as file:
    model = pickle.load(file)
with mlflow.start_run(run_name='AdaBoost_Smote'):
    #model = build_pipeline(algo_ml=model)
    #model.fit(train_sample_set.drop(columns=['SK_ID_CURR']), train_sample_labels)

    # Prédictions sur les données d'entraînement
    #train_preds = model.predict(train_sample_set.drop(columns=['SK_ID_CURR']))
    # Prédictions sur les données de validation
    val_preds = model.predict(val_sample_set.drop(columns=['TARGET']))

    # Calcul des métriques
    roc_auc = roc_auc_score(val_sample_labels, val_preds)
    recall = recall_score(val_sample_labels, val_preds)
    precision = precision_score(val_sample_labels, val_preds)
    f1 = f1_score(val_sample_labels, val_preds)
    cout = cout_metier(val_sample_labels, val_preds)


    # Enregistrement des métriques dans les logs MLflow
    mlflow.log_metric("roc_auc", roc_auc)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("f1_score", f1)
    mlflow.log_metric("cout", cout)

    print(roc_auc)
    print(recall)
    print(precision)
    print(f1)
    print(cout)

mlflow.end_run()

0.5160987155312156
0.03825136612021858
0.3888888888888889
0.06965174129353234
0.6738653001464129


In [46]:
mlflow.set_experiment("AdaBoost")
mlflow.sklearn.autolog()  # Ici on fait appel à l'API MLflow qui intègre les algos de Scikit-Learn.
                          # C'est grâce à ce code qu'on enregistre toutes les métriques et métadonnées de cette expérience (Module MLflow Tracking)

with open('/content/drive/MyDrive/Projet 7/Adaboost/model_0.pkl', 'rb') as file:
    model = pickle.load(file)
with mlflow.start_run(run_name='AdaBoost_wight'):
    #model = build_pipeline(algo_ml=model)
    #model.fit(train_sample_set.drop(columns=['SK_ID_CURR']), train_sample_labels)

    # Prédictions sur les données d'entraînement
    #train_preds = model.predict(train_sample_set.drop(columns=['SK_ID_CURR']))
    # Prédictions sur les données de validation
    val_preds = model.predict(val_sample_set.drop(columns=['TARGET']))

    # Calcul des métriques
    roc_auc = roc_auc_score(val_sample_labels, val_preds)
    recall = recall_score(val_sample_labels, val_preds)
    precision = precision_score(val_sample_labels, val_preds)
    f1 = f1_score(val_sample_labels, val_preds)
    cout = cout_metier(val_sample_labels, val_preds)


    # Enregistrement des métriques dans les logs MLflow
    mlflow.log_metric("roc_auc", roc_auc)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("f1_score", f1)
    mlflow.log_metric("cout", cout)

    print(roc_auc)
    print(recall)
    print(precision)
    print(f1)
    print(cout)

mlflow.end_run()

0.5
0.0
0.0
0.0
0.6650805270863837


Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.


In [50]:
mlflow.set_experiment("AdaBoost")
mlflow.sklearn.autolog()  # Ici on fait appel à l'API MLflow qui intègre les algos de Scikit-Learn.
                          # C'est grâce à ce code qu'on enregistre toutes les métriques et métadonnées de cette expérience (Module MLflow Tracking)

with open('/content/drive/MyDrive/Projet 7/Adaboost/model_us.pkl', 'rb') as file:
    model = pickle.load(file)
with mlflow.start_run(run_name='AdaBoost_Undersampling'):
    #model = build_pipeline(algo_ml=model)
    #model.fit(train_sample_set.drop(columns=['SK_ID_CURR']), train_sample_labels)

    # Prédictions sur les données d'entraînement
    #train_preds = model.predict(train_sample_set.drop(columns=['SK_ID_CURR']))
    # Prédictions sur les données de validation
    val_preds = model.predict(val_sample_set.drop(columns=['TARGET']))

    # Calcul des métriques
    roc_auc = roc_auc_score(val_sample_labels, val_preds)
    recall = recall_score(val_sample_labels, val_preds)
    precision = precision_score(val_sample_labels, val_preds)
    f1 = f1_score(val_sample_labels, val_preds)
    cout = cout_metier(val_sample_labels, val_preds)


    # Enregistrement des métriques dans les logs MLflow
    mlflow.log_metric("roc_auc", roc_auc)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("f1_score", f1)
    mlflow.log_metric("cout", cout)

    print(roc_auc)
    print(recall)
    print(precision)
    print(f1)
    print(cout)

mlflow.end_run()

0.5013563461058431
0.00546448087431694
0.16666666666666666
0.010582010582010581
0.6650805270863837


In [56]:
mlflow.set_experiment("AdaBoost")
mlflow.sklearn.autolog()  # Ici on fait appel à l'API MLflow qui intègre les algos de Scikit-Learn.
                          # C'est grâce à ce code qu'on enregistre toutes les métriques et métadonnées de cette expérience (Module MLflow Tracking)

with open('/content/drive/MyDrive/Projet 7/Adaboost/model_su02.pkl', 'rb') as file:
    model = pickle.load(file)
with mlflow.start_run(run_name='AdaBoost_SMOTE_02'):
    #model = build_pipeline(algo_ml=model)
    #model.fit(train_sample_set.drop(columns=['SK_ID_CURR']), train_sample_labels)

    # Prédictions sur les données d'entraînement
    #train_preds = model.predict(train_sample_set.drop(columns=['SK_ID_CURR']))
    # Prédictions sur les données de validation
    val_preds = model.predict(val_sample_set.drop(columns=['TARGET']))

    # Calcul des métriques
    roc_auc = roc_auc_score(val_sample_labels, val_preds)
    recall = recall_score(val_sample_labels, val_preds)
    precision = precision_score(val_sample_labels, val_preds)
    f1 = f1_score(val_sample_labels, val_preds)
    cout = cout_metier(val_sample_labels, val_preds)


    # Enregistrement des métriques dans les logs MLflow
    mlflow.log_metric("roc_auc", roc_auc)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("f1_score", f1)
    mlflow.log_metric("cout", cout)

    print(roc_auc)
    print(recall)
    print(precision)
    print(f1)
    print(cout)

mlflow.end_run()

0.5
0.0
0.0
0.0
0.6650805270863837


Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.


In [59]:
mlflow.set_experiment("AdaBoost")
mlflow.sklearn.autolog()  # Ici on fait appel à l'API MLflow qui intègre les algos de Scikit-Learn.
                          # C'est grâce à ce code qu'on enregistre toutes les métriques et métadonnées de cette expérience (Module MLflow Tracking)

with open('/content/drive/MyDrive/Projet 7/Adaboost/model_su04.pkl', 'rb') as file:
    model = pickle.load(file)
with mlflow.start_run(run_name='AdaBoost_SMOTE_04'):
    #model = build_pipeline(algo_ml=model)
    #model.fit(train_sample_set.drop(columns=['SK_ID_CURR']), train_sample_labels)

    # Prédictions sur les données d'entraînement
    #train_preds = model.predict(train_sample_set.drop(columns=['SK_ID_CURR']))
    # Prédictions sur les données de validation
    val_preds = model.predict(val_sample_set.drop(columns=['TARGET']))

    # Calcul des métriques
    roc_auc = roc_auc_score(val_sample_labels, val_preds)
    recall = recall_score(val_sample_labels, val_preds)
    precision = precision_score(val_sample_labels, val_preds)
    f1 = f1_score(val_sample_labels, val_preds)
    cout = cout_metier(val_sample_labels, val_preds)


    # Enregistrement des métriques dans les logs MLflow
    mlflow.log_metric("roc_auc", roc_auc)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("f1_score", f1)
    mlflow.log_metric("cout", cout)

    print(roc_auc)
    print(recall)
    print(precision)
    print(f1)
    print(cout)

mlflow.end_run()

0.5
0.0
0.0
0.0
0.6650805270863837


Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.


In [27]:
!pip install pyngrok

In [53]:

from pyngrok import ngrok
# Terminate open tunnels if exist
ngrok.kill()

# Setting the authtoken (optional)
# Get your authtoken from https://dashboard.ngrok.com/auth
NGROK_AUTH_TOKEN = "2YGm2G54kBEMvnAj43IFcf4QK1O_7kd3J9UnbA4TGcvw9W5Np"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Open an HTTPs tunnel on port 5000 for http://localhost:5000
ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)
print("MLflow Tracking UI:", ngrok_tunnel.public_url)

MLflow Tracking UI: https://1f58-35-186-169-213.ngrok-free.app


In [54]:
!mlflow ui

[2023-12-05 19:36:16 +0000] [17466] [INFO] Starting gunicorn 21.2.0
[2023-12-05 19:36:16 +0000] [17466] [INFO] Listening at: http://127.0.0.1:5000 (17466)
[2023-12-05 19:36:16 +0000] [17466] [INFO] Using worker: sync
[2023-12-05 19:36:16 +0000] [17467] [INFO] Booting worker with pid: 17467
[2023-12-05 19:36:16 +0000] [17468] [INFO] Booting worker with pid: 17468
[2023-12-05 19:36:16 +0000] [17469] [INFO] Booting worker with pid: 17469
[2023-12-05 19:36:16 +0000] [17470] [INFO] Booting worker with pid: 17470

[2023-12-05 19:36:56 +0000] [17466] [INFO] Handling signal: int
Aborted!
[2023-12-05 19:36:57 +0000] [17468] [INFO] Worker exiting (pid: 17468)
[2023-12-05 19:36:57 +0000] [17469] [INFO] Worker exiting (pid: 17469)
[2023-12-05 19:36:57 +0000] [17467] [INFO] Worker exiting (pid: 17467)
[2023-12-05 19:36:57 +0000] [17470] [INFO] Worker exiting (pid: 17470)
[2023-12-05 19:36:58 +0000] [17466] [INFO] Shutting down: Master
